In [11]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
import numpy as np

from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, CSVLogger
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa

In [2]:
print(tf.version.VERSION)

2.5.0


In [3]:
#define our RCCNet model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(27, 27, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 27, 27, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 27, 27, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 25, 25, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 12, 12, 64)        2

In [8]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), cooldown=0, patience=6, min_lr=0.5e-15)
csv_logger = CSVLogger('RCCNet_F1score_NoDA_dummy.csv', append=True, separator=';')

In [13]:
adam = Adam(lr=0.00006, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1.0e-6,amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[tf.keras.metrics.CategoricalAccuracy(),
                           tfa.metrics.F1Score(num_classes=4, average='macro')])

In [15]:
TRAIN_DIR = 'data/cell_type/training'
VAL_DIR = 'data/cell_type/testing'
INPUT_WIDTH = 27
INPUT_HEIGHT = 27

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(INPUT_WIDTH,INPUT_HEIGHT),
    class_mode='categorical',
    batch_size=128
)

validation_generator = validation_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(INPUT_WIDTH,INPUT_HEIGHT),
    class_mode='categorical',
    batch_size=64
)

Found 7915 images belonging to 4 classes.
Found 1981 images belonging to 4 classes.


In [17]:
#Early stopping when training_acc > 0.99
# class myCallBack(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}):
#         if logs.get('accuracy') >= 0.99:
#             print('\nReached 99% accuracy so cancelling training!')
#             self.model.stop_training = True

history = model.fit(train_generator, 
                    epochs=500, 
                    #steps_per_epoch=(total_training_imgs // 128), 
                    validation_data = validation_generator, 
                    verbose = 1,
#                     callbacks=[myCallBack()]
                    #validation_steps=(total_val_imgs // 64)
)

Epoch 1/500
62/62 [==============================] - 14s 220ms/step - loss: 1.0683 - categorical_accuracy: 0.6136 - f1_score: 0.5460 - val_loss: 1.8616 - val_categorical_accuracy: 0.4119 - val_f1_score: 0.1459
Epoch 2/500
62/62 [==============================] - 14s 220ms/step - loss: 0.9880 - categorical_accuracy: 0.6310 - f1_score: 0.5621 - val_loss: 2.2658 - val_categorical_accuracy: 0.4119 - val_f1_score: 0.1459
Epoch 3/500
62/62 [==============================] - 13s 201ms/step - loss: 0.9253 - categorical_accuracy: 0.6601 - f1_score: 0.5894 - val_loss: 2.2816 - val_categorical_accuracy: 0.4119 - val_f1_score: 0.1463
Epoch 4/500
62/62 [==============================] - 12s 199ms/step - loss: 0.9176 - categorical_accuracy: 0.6594 - f1_score: 0.5893 - val_loss: 2.5692 - val_categorical_accuracy: 0.4119 - val_f1_score: 0.1461
Epoch 5/500
62/62 [==============================] - 12s 197ms/step - loss: 0.8635 - categorical_accuracy: 0.6792 - f1_score: 0.6144 - val_loss: 1.9941 - val_ca

Epoch 40/500
62/62 [==============================] - 12s 195ms/step - loss: 0.6377 - categorical_accuracy: 0.7522 - f1_score: 0.6959 - val_loss: 0.5696 - val_categorical_accuracy: 0.7875 - val_f1_score: 0.7265
Epoch 41/500
62/62 [==============================] - 12s 196ms/step - loss: 0.6324 - categorical_accuracy: 0.7522 - f1_score: 0.6966 - val_loss: 0.5503 - val_categorical_accuracy: 0.7900 - val_f1_score: 0.7415
Epoch 42/500
62/62 [==============================] - 12s 194ms/step - loss: 0.6289 - categorical_accuracy: 0.7586 - f1_score: 0.7056 - val_loss: 0.5931 - val_categorical_accuracy: 0.7617 - val_f1_score: 0.7223
Epoch 43/500
62/62 [==============================] - 12s 196ms/step - loss: 0.6210 - categorical_accuracy: 0.7572 - f1_score: 0.7042 - val_loss: 0.5650 - val_categorical_accuracy: 0.7819 - val_f1_score: 0.7433
Epoch 44/500
62/62 [==============================] - 12s 195ms/step - loss: 0.6210 - categorical_accuracy: 0.7582 - f1_score: 0.7066 - val_loss: 0.5511 - v

Epoch 79/500
62/62 [==============================] - 12s 195ms/step - loss: 0.5360 - categorical_accuracy: 0.7985 - f1_score: 0.7537 - val_loss: 0.5409 - val_categorical_accuracy: 0.8031 - val_f1_score: 0.7502
Epoch 80/500
62/62 [==============================] - 12s 198ms/step - loss: 0.5453 - categorical_accuracy: 0.7929 - f1_score: 0.7486 - val_loss: 0.5172 - val_categorical_accuracy: 0.8031 - val_f1_score: 0.7590
Epoch 81/500
62/62 [==============================] - 12s 197ms/step - loss: 0.5425 - categorical_accuracy: 0.7931 - f1_score: 0.7490 - val_loss: 0.5360 - val_categorical_accuracy: 0.7920 - val_f1_score: 0.7543
Epoch 82/500
62/62 [==============================] - 12s 195ms/step - loss: 0.5307 - categorical_accuracy: 0.7958 - f1_score: 0.7534 - val_loss: 0.5238 - val_categorical_accuracy: 0.7966 - val_f1_score: 0.7524
Epoch 83/500
62/62 [==============================] - 12s 195ms/step - loss: 0.5400 - categorical_accuracy: 0.7939 - f1_score: 0.7530 - val_loss: 0.5109 - v

Epoch 118/500
62/62 [==============================] - 12s 195ms/step - loss: 0.4910 - categorical_accuracy: 0.8111 - f1_score: 0.7719 - val_loss: 0.4689 - val_categorical_accuracy: 0.8243 - val_f1_score: 0.7817
Epoch 119/500
62/62 [==============================] - 12s 197ms/step - loss: 0.4858 - categorical_accuracy: 0.8163 - f1_score: 0.7787 - val_loss: 0.4632 - val_categorical_accuracy: 0.8233 - val_f1_score: 0.7786
Epoch 120/500
62/62 [==============================] - 12s 197ms/step - loss: 0.4870 - categorical_accuracy: 0.8188 - f1_score: 0.7806 - val_loss: 0.5083 - val_categorical_accuracy: 0.8107 - val_f1_score: 0.7679
Epoch 121/500
62/62 [==============================] - 12s 197ms/step - loss: 0.4920 - categorical_accuracy: 0.8125 - f1_score: 0.7757 - val_loss: 0.4831 - val_categorical_accuracy: 0.8203 - val_f1_score: 0.7825
Epoch 122/500
62/62 [==============================] - 12s 197ms/step - loss: 0.4769 - categorical_accuracy: 0.8201 - f1_score: 0.7816 - val_loss: 0.474

Epoch 157/500
62/62 [==============================] - 12s 199ms/step - loss: 0.4493 - categorical_accuracy: 0.8279 - f1_score: 0.7920 - val_loss: 0.4527 - val_categorical_accuracy: 0.8248 - val_f1_score: 0.7881
Epoch 158/500
62/62 [==============================] - 13s 201ms/step - loss: 0.4591 - categorical_accuracy: 0.8264 - f1_score: 0.7912 - val_loss: 0.4594 - val_categorical_accuracy: 0.8304 - val_f1_score: 0.7908
Epoch 159/500
62/62 [==============================] - 13s 204ms/step - loss: 0.4504 - categorical_accuracy: 0.8293 - f1_score: 0.7943 - val_loss: 0.4700 - val_categorical_accuracy: 0.8218 - val_f1_score: 0.7768
Epoch 160/500
62/62 [==============================] - 13s 202ms/step - loss: 0.4479 - categorical_accuracy: 0.8318 - f1_score: 0.7986 - val_loss: 0.5795 - val_categorical_accuracy: 0.7855 - val_f1_score: 0.7393
Epoch 161/500
62/62 [==============================] - 13s 201ms/step - loss: 0.4476 - categorical_accuracy: 0.8296 - f1_score: 0.7950 - val_loss: 0.496

...

Epoch 313/500
62/62 [==============================] - 12s 196ms/step - loss: 0.3554 - categorical_accuracy: 0.8710 - f1_score: 0.8464 - val_loss: 0.4632 - val_categorical_accuracy: 0.8274 - val_f1_score: 0.7867
Epoch 314/500
62/62 [==============================] - 12s 195ms/step - loss: 0.3554 - categorical_accuracy: 0.8719 - f1_score: 0.8459 - val_loss: 0.5192 - val_categorical_accuracy: 0.8092 - val_f1_score: 0.7634
Epoch 315/500
62/62 [==============================] - 12s 194ms/step - loss: 0.3558 - categorical_accuracy: 0.8705 - f1_score: 0.8465 - val_loss: 0.5170 - val_categorical_accuracy: 0.8152 - val_f1_score: 0.7640
Epoch 316/500
62/62 [==============================] - 12s 196ms/step - loss: 0.3464 - categorical_accuracy: 0.8665 - f1_score: 0.8402 - val_loss: 0.4464 - val_categorical_accuracy: 0.8344 - val_f1_score: 0.7985
Epoch 317/500
62/62 [==============================] - 12s 197ms/step - loss: 0.3460 - categorical_accuracy: 0.8718 - f1_score: 0.8454 - val_loss: 0.512

Epoch 352/500
62/62 [==============================] - 12s 196ms/step - loss: 0.3432 - categorical_accuracy: 0.8706 - f1_score: 0.8444 - val_loss: 0.4763 - val_categorical_accuracy: 0.8314 - val_f1_score: 0.7877
Epoch 353/500
62/62 [==============================] - 12s 195ms/step - loss: 0.3357 - categorical_accuracy: 0.8743 - f1_score: 0.8495 - val_loss: 0.4657 - val_categorical_accuracy: 0.8304 - val_f1_score: 0.7884
Epoch 354/500
62/62 [==============================] - 12s 195ms/step - loss: 0.3415 - categorical_accuracy: 0.8735 - f1_score: 0.8476 - val_loss: 0.4771 - val_categorical_accuracy: 0.8233 - val_f1_score: 0.7808
Epoch 355/500
62/62 [==============================] - 12s 196ms/step - loss: 0.3393 - categorical_accuracy: 0.8738 - f1_score: 0.8490 - val_loss: 0.4713 - val_categorical_accuracy: 0.8243 - val_f1_score: 0.7886
Epoch 356/500
62/62 [==============================] - 12s 196ms/step - loss: 0.3405 - categorical_accuracy: 0.8744 - f1_score: 0.8499 - val_loss: 0.468

Epoch 391/500
62/62 [==============================] - 12s 186ms/step - loss: 0.3212 - categorical_accuracy: 0.8764 - f1_score: 0.8527 - val_loss: 0.4771 - val_categorical_accuracy: 0.8304 - val_f1_score: 0.7888
Epoch 392/500
62/62 [==============================] - 12s 186ms/step - loss: 0.3296 - categorical_accuracy: 0.8748 - f1_score: 0.8514 - val_loss: 0.4659 - val_categorical_accuracy: 0.8390 - val_f1_score: 0.8023
Epoch 393/500
62/62 [==============================] - 12s 186ms/step - loss: 0.3265 - categorical_accuracy: 0.8768 - f1_score: 0.8526 - val_loss: 0.5057 - val_categorical_accuracy: 0.8193 - val_f1_score: 0.7673
Epoch 394/500
62/62 [==============================] - 12s 186ms/step - loss: 0.3274 - categorical_accuracy: 0.8802 - f1_score: 0.8579 - val_loss: 0.5170 - val_categorical_accuracy: 0.8082 - val_f1_score: 0.7645
Epoch 395/500
62/62 [==============================] - 12s 187ms/step - loss: 0.3275 - categorical_accuracy: 0.8778 - f1_score: 0.8532 - val_loss: 0.506

Epoch 430/500
62/62 [==============================] - 11s 177ms/step - loss: 0.3149 - categorical_accuracy: 0.8801 - f1_score: 0.8573 - val_loss: 0.4925 - val_categorical_accuracy: 0.8193 - val_f1_score: 0.7791
Epoch 431/500
62/62 [==============================] - 11s 179ms/step - loss: 0.3278 - categorical_accuracy: 0.8769 - f1_score: 0.8519 - val_loss: 0.4903 - val_categorical_accuracy: 0.8147 - val_f1_score: 0.7739
Epoch 432/500
62/62 [==============================] - 11s 179ms/step - loss: 0.3164 - categorical_accuracy: 0.8830 - f1_score: 0.8589 - val_loss: 0.5151 - val_categorical_accuracy: 0.8208 - val_f1_score: 0.7762
Epoch 433/500
62/62 [==============================] - 11s 180ms/step - loss: 0.3121 - categorical_accuracy: 0.8807 - f1_score: 0.8579 - val_loss: 0.4814 - val_categorical_accuracy: 0.8264 - val_f1_score: 0.7885
Epoch 434/500
62/62 [==============================] - 11s 178ms/step - loss: 0.3169 - categorical_accuracy: 0.8816 - f1_score: 0.8580 - val_loss: 0.476

Epoch 469/500
62/62 [==============================] - 12s 195ms/step - loss: 0.2892 - categorical_accuracy: 0.8925 - f1_score: 0.8713 - val_loss: 0.5000 - val_categorical_accuracy: 0.8243 - val_f1_score: 0.7885
Epoch 470/500
62/62 [==============================] - 12s 194ms/step - loss: 0.2975 - categorical_accuracy: 0.8910 - f1_score: 0.8690 - val_loss: 0.4825 - val_categorical_accuracy: 0.8304 - val_f1_score: 0.7868
Epoch 471/500
62/62 [==============================] - 12s 195ms/step - loss: 0.2994 - categorical_accuracy: 0.8874 - f1_score: 0.8651 - val_loss: 0.4915 - val_categorical_accuracy: 0.8228 - val_f1_score: 0.7814
Epoch 472/500
62/62 [==============================] - 12s 194ms/step - loss: 0.2997 - categorical_accuracy: 0.8874 - f1_score: 0.8661 - val_loss: 0.4684 - val_categorical_accuracy: 0.8364 - val_f1_score: 0.7970
Epoch 473/500
62/62 [==============================] - 12s 196ms/step - loss: 0.3056 - categorical_accuracy: 0.8863 - f1_score: 0.8654 - val_loss: 0.478